In [1]:
# Advent of Code 2023
# Day 10 Problem 1
import re
from collections import Counter
import functools

with open("aoc_10_input.txt") as f:
    A = f.read().strip().split("\n")

# c+p test case here
TEST = """..F7.
.FJ|.
SJ.L7
|F--J
LJ...""".strip().split("\n")

TESTB = """7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ""".strip().split("\n")

for i, line in enumerate(TEST):
    print(f"{i}: {line}")


0: ..F7.
1: .FJ|.
2: SJ.L7
3: |F--J
4: LJ...


In [2]:
# my first attempt is recursive and hits the recursion limit
# so I will reimplement as an iterative solution.
# however, these three are used between both attempts
def parseInput(lines):
    L = len(lines[0])
    res = [["." for _ in range(L+2)]]
    for line in lines:
        temp = list(line)
        temp.append(".")
        temp.insert(0, ".")
        res.append(temp)
    res.append(["." for _ in range(L+2)])
    return res

def findS(inp):
    for i, row in enumerate(inp):
        for j, char in enumerate(row):
            if char == "S":
                return (i,j)
    return (-1,-1)

def findNeighbors(M, x, y):
    res = []
    # north
    if M[x-1][y] in "|7F":
        res.append([x-1, y])
    # south
    if M[x+1][y] in "|LJ":
        res.append([x+1, y])
    # west
    if M[x][y-1] in "-LF":
        res.append([x, y-1])
    # east
    if M[x][y+1] in "-7J":
        res.append([x, y+1])
    return res


In [3]:
# these are not used in my solution
def goDown(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "|":
        return goDown(M,x+1,y,count+1)
    if M[x][y] == "L":
        return goRight(M,x,y+1,count+1)
    if M[x][y] == "J":
        return goLeft(M,x,y-1,count+1)
    return 0

def goUp(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "|":
        return goUp(M,x-1,y,count+1)
    if M[x][y] == "F":
        return goRight(M,x,y+1,count+1)
    if M[x][y] == "7":
        return goLeft(M,x,y-1,count+1)
    return 0

def goRight(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "-":
        return goRight(M,x,y+1,count+1)
    if M[x][y] == "7":
        return goDown(M,x+1,y,count+1)
    if M[x][y] == "J":
        return goUp(M,x-1,y,count+1)
    return 0

def goLeft(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "-":
        return goLeft(M,x,y-1,count+1)
    if M[x][y] == "F":
        return goDown(M,x+1,y,count+1)
    if M[x][y] == "L":
        return goUp(M,x-1,y,count+1)
    return 0

def initialDirection(M, sx, sy, nx, ny):
    # same row
    if sx == nx:
        # west
        if ny == sy - 1:
            print("Go left!")
            return goLeft(M,nx,ny,1)
        # east
        elif ny == sy + 1:
            print("Go right!")
            return goRight(M,nx,ny,1)
    # same col
    else:
        # north
        if nx == sx - 1:
            print("Go up!")
            return goUp(M,nx,ny,1)
        elif nx == sx + 1:
            print("Go down!")
            return goDown(M,nx,ny,1)
    return 0

# this hits the recursion limit so I will try out an iterative version instead
def p1(inp):
    tapestry = parseInput(inp)
    #for row in tapestry:
    #    print(row)
    START = findS(tapestry)
    print(f"Starting point is at {START}\n")
    startNeighbors = findNeighbors(tapestry, *START)
    print(f"Starting neighbors: {startNeighbors}")
    res = initialDirection(tapestry, *START, *startNeighbors[0])
    print(f"{res}//2 = {res//2}")
    return res//2


In [4]:
def traverse(M, stack):
    while(stack[-1] != stack[0]):
        px, py = stack[-1][0], stack[-1][1]
        diff = [px - stack[-2][0], py - stack[-2][1]]
        # we are going south
        if diff == [1, 0]:
            if M[px][py] == "|":        nextPos = [px+1, py]    # continue going south
            elif M[px][py] == "L":      nextPos = [px, py+1]    # go east
            elif M[px][py] == "J":      nextPos = [px, py-1]    # go west           
        # we are going north
        elif diff == [-1,0]:
            if M[px][py] == "|":        nextPos = [px-1, py]    # continue going north
            elif M[px][py] == "F":      nextPos = [px, py+1]    # go east
            elif M[px][py] == "7":      nextPos = [px, py-1]    # go west
        # we are going east
        elif diff == [0,1]:
            if M[px][py] == "-":        nextPos = [px, py+1]    # continue going east
            elif M[px][py] == "J":      nextPos = [px-1, py]    # go north
            elif M[px][py] == "7":      nextPos = [px+1, py]    # go south
        # we are going west
        elif diff == [0,-1]:
            if M[px][py] == "-":        nextPos = [px, py-1]    # continue going west
            elif M[px][py] == "L":      nextPos = [px-1, py]    # go north
            elif M[px][py] == "F":      nextPos = [px+1, py]    # go south            
        stack.append(nextPos)
    return stack[:-1]

def p1_take2(inp):
    tapestry = parseInput(inp)
    startPos = findS(tapestry)
    startNeighbors = findNeighbors(tapestry, *startPos)
    stack = [list(startPos), startNeighbors[0]]
    completePath = traverse(tapestry, stack)
    return len(completePath)//2


In [5]:
# expected output: 8
#print(p1(TEST))
print(p1_take2(TESTB))



8


In [6]:
p1_take2(A)


6890

In [7]:
# Part 2
# So all the solutions I can think of don't account for 
# the paths "between" the pipes. So if we expand the world
# where each single point becomes a 3x3 grid, we can then do
# floodOutside() and have it work this time. 

def expandMap(tap, loop):
    tiles = {
        ".": ["...","...","..."],
        "|": [".#.",".#.",".#."],
        "-": ["...","###","..."],
        "F": ["...",".##",".#."],
        "7": ["...","##.",".#."],
        "J": [".#.","##.","..."],
        "L": [".#.",".##","..."]
    }
    res = []
    for row in tap:
        t, m, b = "", "", ""
        for c in row:
            M = tiles[c]
            t += M[0]
            m += M[1]
            b += M[2]
        res.append(t)
        res.append(m)
        res.append(b)
    return res
            

def transformTapestry(M, S):
    res = []
    for i,row in enumerate(M):
        r = []
        for j,node in enumerate(row):
            if [i,j] in S:
                r.append(" ")
            else:
                r.append("o")
        res.append(r)
    return floodOutside(res)

def floodOutside(tap):
    queue = [[0,0]]
    H, W = len(tap)-1, len(tap[0])-1
    while queue:
        nx,ny = queue.pop()
        tap[nx] = tap[nx][:ny] + " " + tap[nx][ny+1:]
        if nx > 0 and tap[nx-1][ny] == ".":     queue.append([nx-1,ny])
        if ny > 0 and tap[nx][ny-1] == ".":     queue.append([nx,ny-1])
        if nx < H and tap[nx+1][ny] == ".":     queue.append([nx+1,ny])
        if ny < W and tap[nx][ny+1] == ".":     queue.append([nx,ny+1])
    return tap

def replaceS(tap, a, b):
    ax, ay = a
    bx, by = b
    if tap[ax][ay] in "-J7" and tap[bx][by] in "|JL":   return "F"
    if tap[ax][ay] in "-LF" and tap[bx][by] in "|JL":   return "7"
    if tap[ax][ay] in "-LF" and tap[bx][by] in "-J7":   return "-"
    if tap[ax][ay] in "|7F" and tap[bx][by] in "-J7":   return "L"
    if tap[ax][ay] in "|7F" and tap[bx][by] in "|JL":   return "|"
    if tap[ax][ay] in "|7F" and tap[bx][by] in "-LF":   return "J"
    return "."

def yieldPipeMap(tapestry, path):
    res = []
    for i,row in enumerate(tapestry):
        r = ""
        for j,v in enumerate(row):
            if [i,j] in path:   r += v
            else:               r += "."
        res.append(r)
    return res

def pMap(m):
    for row in m:
        print("".join(row))

def checkInterior(M):
    # since we exploded everything by 3, everything should be divisible by 3
    H, W = len(M)//3, len(M[0])//3
    res = 0
    for i in range(H):
        for j in range(W):   
            ti, tj = 3 * i, 3 * j     
            if all([M[ti][tj:tj+3] == "...", M[ti+1][tj:tj+3] == "...", M[ti+2][tj:tj+3] == "..."]):
                res += 1
    return res

def p2(inp):
    tapestry = parseInput(inp)
    startPos = findS(tapestry)
    #pMap(tapestry)

    startNeighbors = sorted(findNeighbors(tapestry, *startPos))
    newS = replaceS(tapestry, *startNeighbors)
    tapestry[startPos[0]][startPos[1]] = newS
    #print(f"Starting point is at {startPos}")
    #print(f"Starting neighbors: {startNeighbors}")
    #print(f"S should be replaced with a {newS}\n")

    stack = [list(startPos), startNeighbors[0]]
    completePath = traverse(tapestry, stack)
    pipeMap = yieldPipeMap(tapestry, completePath)
    #pMap(pipeMap)

    explodedMap = expandMap(pipeMap, completePath)
    #pMap(explodedMap)

    flooded = floodOutside(explodedMap)
    #pMap(flooded)

    return checkInterior(flooded)

In [8]:
TESTC = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L""".strip().split("\n")


In [9]:
p2(TESTC)

10

In [10]:
p2(A)

453